In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

import os

# Configuration

"Data release" refers to https://www.sciencebase.gov/catalog/item/5e5c1b4fe4b01d50924f27e1

In [2]:
ice_obs_fpath = "../in/MN_ice/ice_duration_summarized.csv"

prior_data_release_dir = "../in/prior_data_release/"
data_release_metadata_fpath = prior_data_release_dir + "lake_metadata.csv"

out_dir = "../out/"
out_driver_dir = out_dir + "merged_drivers/by_DOW/"
matching_df_fpath = out_dir + "matching_sources.csv"
missing_pgdl_fpath = out_dir + "DOW_missing_PGDL_estimates.npy"

# Import base data sets

In [3]:
ice_dur = pd.read_csv(ice_obs_fpath)
data_release_metadata = pd.read_csv(data_release_metadata_fpath)

In [4]:
# how many unique DOWs exist in the ice targets data set?
ice_dur['DOW'].unique().shape

(1156,)

### Create a new data set for easily referencing between the observations and drivers

The main usefulness here is that the ice observations are provided by Minnesota DOW identifiers while the TOHA data release drivers are primarily sorted by NHD identifiers. Sometimes one NHD identifier corresponds to multiple DOW identifiers, so this code unravels that duplication (i.e., individual rows rather than rows with list values) - being more repetitive but more easily accessible from the DOW point of view.

In [5]:
for val in data_release_metadata['site_id']:
    assert(val[:6] == 'nhdhr_') # check consistent format
    remaining_val = val[6:]
    assert(type(remaining_val) == str) # not list
    
DOW_nums_ls = []
group_ls = []
meteo_file_ls = []
nhdhr_ls = []

for DOW in data_release_metadata['mndow_id']:
    DOW_nums = DOW.replace("|", "").split('mndow_')[1:]
    # unnesting the multiple lake aggregating and adding rows for them (and their files)
    for ID in DOW_nums:
        DOW_nums_ls.append(int(ID))
        group_ls.append(data_release_metadata[data_release_metadata['mndow_id'] == DOW]['group_id'].item())
        meteo_file_ls.append(data_release_metadata[data_release_metadata['mndow_id'] == DOW]['meteo_filename'].item())
        nhdhr_ls.append(data_release_metadata[data_release_metadata['mndow_id'] == DOW]['site_id'].item()[6:])
        
matching_df = pd.DataFrame({'DOW':DOW_nums_ls,
                            'group':group_ls,
                            'meteo_file':meteo_file_ls,
                            'nhdhr':nhdhr_ls})
matching_df

,DOW,group,meteo_file,nhdhr
0,3065700,06_N46.00-47.00_W94.50-97.00,nldas_meteo_N46.8125-46.8125_W96.1875-96.1875.csv,121545300
1,6015200,09_N45.00-46.00_W94.50-97.00,nldas_meteo_N45.4375-45.4375_W96.5625-96.5625.csv,122548488
2,37004600,09_N45.00-46.00_W94.50-97.00,nldas_meteo_N45.0625-45.0625_W95.9375-95.9375.csv,122551004
3,37004601,09_N45.00-46.00_W94.50-97.00,nldas_meteo_N45.0625-45.0625_W95.9375-95.9375.csv,122551004
4,3029100,06_N46.00-47.00_W94.50-97.00,nldas_meteo_N46.9375-46.9375_W95.8125-95.8125.csv,121544299
...,...,...,...,...
1058,69027700,04_N45.50-48.00_W92.00-93.00,nldas_meteo_N47.8125-47.8125_W92.0625-92.0625.csv,105954667
1059,38062000,01_N48.00-49.50_W89.50-97.25,nldas_meteo_N48.0625-48.0625_W91.4375-91.4375.csv,80997051
1060,38014700,01_N48.00-49.50_W89.50-97.25,nldas_meteo_N48.0625-48.0625_W91.0625-91.0625.csv,80997393
1061,16081200,01_N48.00-49.50_W89.50-97.25,nldas_meteo_N48.0625-48.0625_W91.0625-91.0625.csv,80994457


In [6]:
# save it for later use
matching_df.to_csv(matching_df_fpath)

# For each DOW identifier, aggregrate and store all the data release drivers

In [7]:
# inner merge omits any lakes that do not have
# corresponding values in the both data sources
ice_dur = ice_dur.merge(matching_df,
                        how = 'inner',
                        on = 'DOW')

In [8]:
# how many unique DOWs and NHDHRs made it through the merge?
ice_dur['DOW'].unique().shape, ice_dur['nhdhr'].unique().shape

((633,), (581,))

In [9]:
missing_pgdl = []

count = 0
for DOW in ice_dur['DOW'].unique():
    
    # subset the df for value grabbing
    cur_df = ice_dur[ice_dur['DOW'] == DOW].reset_index(drop=True)
    
    group = cur_df['group'].unique().item()
    meteo_file = cur_df['meteo_file'].unique().item()
    nhdhr = cur_df['nhdhr'].unique().item()
    
    # get the meteo inputs
    meteo = pd.read_csv(prior_data_release_dir +
                        'inputs_' +
                        group +
                        '/' +
                        meteo_file)

    # get the clarity inputs
    clarity = pd.read_csv(prior_data_release_dir +
                          'clarity_' +
                          group +
                          '/gam_nhdhr_' +
                          nhdhr + 
                          '_clarity.csv') 

    # get the PB ice flags
    ice_flags = pd.read_csv(prior_data_release_dir +
                            'ice_flags_' +
                            group +
                            '/pb0_nhdhr_' +
                            nhdhr + 
                            '_ice_flags.csv') 

    # get the PB irradiance
    irradiance = pd.read_csv(prior_data_release_dir +
                             'irradiance_' +
                             group +
                             '/pb0_nhdhr_' +
                             nhdhr + 
                             '_irradiance.csv')   

    # get the PB water temps
    PB_water_temps = pd.read_csv(prior_data_release_dir +
                                 'pb0_predictions_' +
                                 group +
                                 '/pb0_nhdhr_' +
                                 nhdhr + 
                                 '_temperatures.csv')

    # get the PGDL water temps WHEN THEY ARE AVAILABLE
    try:
        PGDL_water_temps = pd.read_csv(prior_data_release_dir +
                                       'pgdl_predictions_' +
                                       group + 
                                       '/tmp/pgdl_nhdhr_' +
                                       nhdhr + 
                                       '_temperatures.csv')
    # otherwise proceed without them
    except FileNotFoundError as e:
        missing_pgdl.append(DOW)
        PGDL_water_temps = None

    # merge them all
    meteo = meteo.rename(columns = {'time':'date'})
    inputs = meteo.merge(clarity, on = 'date', how = 'left')
    inputs = inputs.merge(ice_flags, on = 'date', how = 'left')
    inputs = inputs.merge(irradiance, on = 'date', how = 'left')
    inputs = inputs.merge(PB_water_temps, on = 'date', how = 'left')
    try:
        inputs = inputs.merge(PGDL_water_temps, on = 'date', how = 'left')
    except TypeError as e:
        pass

    inputs.to_csv(out_driver_dir + "DOW_" + str(DOW) + "_all_vars.csv")

In [10]:
# How many lakes (that made it through the merge) lacked PGDL estimates?
len(missing_pgdl)

148

In [11]:
np.save(missing_pgdl_fpath, missing_pgdl)

Originally, I merged these by nhdhr, but I ultimately decided on DOW as the identifier because DOW is what we have labels for, so it seems more important to be faithful to the labels than the drivers (which are by nhdhr)

<br><br><br><br><br><br><br><br><br>

# Some Quality Checks

Mostly concerned with PGDL estimates

### Quality checking where we have missing data

Identify all the unique lakes and count how many ice observations each unique lake has

In [12]:
count_ls = []
for DOW in ice_dur['DOW'].unique():
    count_ls.append(ice_dur[ice_dur['DOW'] == DOW]['DOW'].value_counts().item())

In [13]:
new_df = pd.DataFrame({'DOW':ice_dur['DOW'].unique(),
                       'count':count_ls})

Now, match that information with which unique lakes are lacking pgdl driver data

In [14]:
indices = []
for i in range(new_df.shape[0]):
    indices.append(new_df['DOW'][i] in missing_pgdl)

In [15]:
new_df[indices].shape

(148, 2)

In [16]:
np.sum(new_df['count'][indices]), np.sum(new_df['count'][indices]) / ice_dur.shape[0]

(975, 0.19102664576802508)

The unique lakes that lack any PGDL estimates amount to 975 observations, which is 19% of the data set. This pays no attention to the timing of PGDL estimates or observations.

<br><br><br>

### Quality checking when we have missing data

Same procedure as above

BUT, first remove years that don't have any driver data anyways

In [17]:
relevant_years = ice_dur[(ice_dur['min_ice_on_date'] > '1980-01-01') * (ice_dur['min_ice_on_date'] <= '2018-07-01')]

In [18]:
count_ls = []
for DOW in relevant_years['DOW'].unique():
    count_ls.append(relevant_years[relevant_years['DOW'] == DOW]['DOW'].value_counts().item())

In [19]:
new_df = pd.DataFrame({'DOW':relevant_years['DOW'].unique(),
                       'count':count_ls})

In [20]:
np.sum(new_df['count']), (ice_dur.shape[0] - np.sum(new_df['count'])) / ice_dur.shape[0]

(4359, 0.1459639498432602)

Years that are outside the temporal bounds of the driver data represent approximately 15% of the data set alone.

Beyond that...

In [21]:
indices = []
for i in range(new_df.shape[0]):
    indices.append(new_df['DOW'][i] in missing_pgdl)

In [22]:
# what?? it should be 15%
# how can 15% of sequences have non-NA pgdl but 20% of them are missing files
np.sum(new_df['count'][indices]), np.sum(new_df['count'][indices]) / np.sum(new_df['count'])

(873, 0.2002752924982794)

An additional ~20% lack PGDL estimates due to their lakes not having PGDL estimates for any time period.

So ~35% of observations do not have PGDL estimates.